# MA6202: Laboratorio de Ciencia de Datos

**Profesor: Nicolás Caro**

**20/07/2020 - E3 S15**


**Integrantes del grupo 6**: Danner Schlotterbeck, José Díaz V., Francisco Vásquez L.

## Ejercicio 3

El objetivo de esta evaluación es resolver un problema de detección de noticias falsas (*Fake News*) usando herramientas de aprendizaje de máquinas. 

Para lograr una representación numérica de los textos utilizaremos la librería `spaCy` para procesamiento de lenguaje natural. 

**Instalaciones previas**  
Para la ejecución correcta de este notebook puede ser necesario ejecutar los siguientes comandos de instalación:

```python
!pip install spacy tqdm
!pip install -c pytorch torchtext
!python -m spacy download en_core_web_sm
```
**Obs:** Puede usar conda en vez de pip si maneja su librería con esta herramienta. 

Las librerías que se instalan son:
- spacy: ampliamente usada para procesamiento de lenguaje natural. Esta librería posee modelos estadísticos preentrenados como `en_core_web_sm` que será detallado posteriormente.
- tqdm: para mostrar barras de progreso en pantalla.
- torchtext: contiene en herramientas populares de procesamiento de lenguaje natural.

In [ ]:
!pip install spacy tqdm
!pip install -c pytorch torchtext
!python -m spacy download en_core_web_sm

**Librerías**  
En la evaluación, **no** estará permitido usar librerías ni módulos diferentes a los declarados en la siguiente celda.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import spacy
import os
import numpy as np
import pickle as pk

from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report,\
    confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torchtext.data import Field, TabularDataset, BucketIterator
from torchtext.vocab import GloVe

'''
Puede utilizar esta extension si trabaja en colaboratory:
from google.colab import drive
drive.mount('/content/drive')
''';

**Replicabilidad**  
A lo largo de todo el ejercicio llamaremos múltiples veces a la función `np.random.seed`, con la semilla fija en la variable `seed_=300`

In [ ]:
seed_ = 300
np.random.seed(seed_)

**Uso de GPU**  
En este ejercicio se utilizarán modelos que requieren alto poder de computo por lo que se recomienda usar GPU. Recuerde que en **Colaboratory** tiene acceso gratuito a dicho recurso.  

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Preliminares

### Carga de datos


- Compruebe que la siguiente celda coincide con este output:

```
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6335 entries, 8476 to 4330
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   6335 non-null   object
 1   text    6335 non-null   object
 2   label   6335 non-null   object
dtypes: object(3)
memory usage: 198.0+ KB
```

In [ ]:
raw_data_path = 'https://raw.githubusercontent.com/NicoCaro/DataScienceLab/master/ejercicios/ejercicio%203/data/news.csv'
raw_df = pd.read_csv(raw_data_path, index_col=0)
raw_df.info()

**Preprocesamiento**  
El conjunto de datos consta de 3 columnas:
- `title`: contiene el título de la noticia
- `text`: contiene el teto de la noticia
- `label`: contiene las etiquetas `REAL` y `FAKE` que indican si se trata de una noticia verdadera o falsa.

En la siguiente celda se incluye la columna `X` con una concatenación del título y el texto de las noticias, además de la columna `y` como una representación numérica de la columna `label`.

In [ ]:
pro_df = raw_df.copy()
pro_df['y'] = (pro_df['label'] == 'FAKE').astype('int')
pro_df['X'] = pro_df['title'].str.cat(pro_df['text'], sep='. ')

# se eliminan las columnas innecesarias
pro_df = pro_df.reindex(columns=['X', 'y'])
display(pro_df.head())

### Procesamiento de texto
A modo de ejemplo se muestra el procesamiento que se busca aplicar a cada una de las observaciones de la columna `X`.

In [ ]:
x_muestra, y_muestra = pro_df.iloc[3].values.T
print('Noticia falsa' if y_muestra else 'Noticia verdadera', '-' * 72,  sep='\n')
print(x_muestra[:501], '...')

**Definición del modelo `spacy`**  
El modelo estadístico de `spacy` que usaremos en el ejercicio es `english_web_sm`, que consiste en un objeto basado en una red convolucional, preentrenada en un conjunto de datos llamado Ontowords y diseñada para resolver múltiples tareas de procesamiento de lenguaje natural, dento de sus métodos se encuentran rutinas de tokenización y lematización detalladas posteriormente.

Dentro de las funcionalidades que entrega este modelo, hay un subconjunto que no se utilizará en el ejercicio. Para ahorrar tiempo de cómputo, estas funcionalidades son deshabilitadas en el  el argumento `disable`.

**Obs:** Puede ser necesario ejecutar `python -m spacy download en_core_web_sm` para tener acceso a tal modelo de lenguaje.

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['entitry_ruler', 'textcat', 
                                            'entity_linker', 'ner', 'tagger'])
print(type(nlp))

**Tokenización**  

Lo primero para analizar texto es separar el campo de texto en _tokens_. Un _token_ es un segmento significativo del texto. La entrada al tokenizer es un texto unicode, y la salida es un `spacy.tokens.doc.Doc`.


El proceso puede entenderse como:
1. Aplicar el método `str.split(' ')` que entrega una lista de `str`.
2. Verificar si cada uno de los elementos de la lista puede subdividirse:
    1. **Porque se trata de una regla de excepción.** Por ejemplo `don't` debería subdividirse en `do` y `n't`, mientras que `U.K.` no debe subdividirse.
    2. **Porque el elemento contiene prefijos, sufijos o [infijos](https://dle.rae.es/infijo).** Por ejemplo comillas, comas, puntos, etc...
    
Para más detalles ver la documentación de [spacy](https://spacy.io/usage/linguistic-features#tokenization).

Así al generar *tokens* en `x_muestra`, los 20 primeros son: 

In [ ]:
spacy_doc = nlp(x_muestra)
[print(token) for token in spacy_doc[:20]];

**Lematización**  
La [lematización](https://es.wikipedia.org/wiki/Lematizaci%C3%B3n) es el proceso de agrupar las formas flexionadas de una palabra (en plural, en femenino, conjugada, etc), para que puedan analizarse como un solo elemento, identificado por el **lema** de la palabra.

En procesamiento de lenguaje natural a lematización depende de la identificación correcta de la [categoría gramatical](https://es.wikipedia.org/wiki/Categor%C3%ADa_gramatical) (*part of speech*). Algunos ejemplos de categoría gramatical son sustantivo, adjetivo, verbo, advervio, etc...

En `spacy` accedemos al lema mediante el atributo `lemma_`. Así, para los 20 primeros *tokens*:

In [ ]:
[print(token.lemma_) for token in spacy_doc[:20]];

**Stopwords**  
Los *tokens* generados pueden ser clasificados como [palabras vacías](https://es.wikipedia.org/wiki/Palabra_vac%C3%ADa) (*stop words*) que no tienen significado en si mismas. Algunos ejemplos son preposiciones, artículos, pronombres, etc... En procesamiento de lenguaje natural es común eliminarlas.

Para este ejercicio eliminaremos las *palabras vacías* mediante el atributo `is_stop`, además de los *token* que no sean alfanuméricos mediante el atributo `is_alpha`. 

In [ ]:
[print(token) for token in spacy_doc[:20] if (not token.is_stop) and token.is_alpha];

#### Pregunta 1
1. Defina la función `procesa_texto` que reciba como argumento un texto y un modelo de lenguaje `spacy`. La función `procesa_texto` debe entregar un texto con los lemas de cada *token*, eliminando las palabras vacías y los *token* que no sean alfanuméricos, usando el procedimiento recién presentado. El texto obtenido debe separar los lemas por espacios `' '`.
2. Aplique dicha función a la columna `X`, guardando sus resultados en la columna `pro_X`. Como modelo de lenguaje `spacy`, utilice el modelo instanciado anteriormente.  
    **Observaciones**:
    - Es usual que esta operación tome bastante tiempo por lo que recomendamos usar el método `progress_apply` de pandas que permite reportar el progreso de la operación mediante la librería `tqdm`. Para habilitar dicho método de la librería pandas debe primero llamar al método `tqdm.pandas`.
    - Puede ser útil guardar el resultado de esta operación en disco, dado el tiempo que toma repetir la operación. 

In [ ]:
def procesa_texto(in_text, spacy_model):

    out_text = ''
    spacy_doc = spacy_model(in_text)

    for token in spacy_doc:
    if (not token.is_stop) and token.is_alpha:
        out_text += " " + token.text
    return out_text.lstrip()

tqdm.pandas()
pro_df['pro_X'] = pro_df['X'].progress_apply(procesa_texto, args=(nlp,))

pro_df.to_pickle('pro_df.pkl')

3. Cuente el número de lemas en cada observación de la columna `pro_X` y compruebe que obtiene las siguientes estadísticas descriptivas:

```
count    6335.000000
mean      393.084294
std       409.950812
min         2.000000
25%       153.000000
50%       311.000000
75%       515.000000
max      8730.000000
Name: pro_X, dtype: float64
```
*Hint*: puede ser útil el método `pd.Series.str.count`.

In [ ]:
pro_df = pd.read_pickle('pro_df.pkl')

num_lem = pro_df['pro_X'].str.count(' ') + 1
num_lem.describe()

4. Instancie `final_df` como una copia de `pro_df` a la que se le elimina la columna `'X'`. Luego aplique la eliminación de los duplicados `final_df`.   
- Compruebe que obtiene un total de 6,303 filas únicas en `final_df`.


In [ ]:
final_df = pro_df.loc[:, pro_df.columns != 'X'].copy()

final_df.drop_duplicates(inplace=True)
final_df

### Definición de notación
De ahora en adelante denotamos por:
- ***Documento***: cada una de las noticias procesadas del conjunto de datos, es decir, cada una de las observaciones de la columna `'pro_X'`del DataFrame `final_df`.
- ***Corpus***: el conjunto de *documentos* del conjunto de datos, es decir, el conjunto de observaciones de la columna `'pro_X'` del DataFrame `final_df`.
- ***Vocabulario***: al conjunto de *tokens* presentes en el *corpus*.


### Definición de conjuntos del problema
Se separan las muestras en dos conjuntos:
- Conjunto de *entrenamiento union validación* (denotado por `*_full_train`), con el 80% de las observaciones
- Conjunto de *prueba* (denotado por `*_test`), con el 20% de las observaciones.

A su vez el conjunto de *entrenamiento union validación* de subdivide en:
- Conjunto de *entrenamiento* (denotado por `*_train`), con el 64% de las observaciones.
- Conjunto de *validación* (denotado por `*_val`), con el 16% de las observaciones

Para realizar esta subdivisión utilizamos dos veces la función `sklearn.model_selection.train_test_split`.

In [ ]:
# define parametros de la division de conjuntos
proporcion_total_entrenamiento_prueba = 0.80
proporcion_entrenamiento_validacion = 0.80

# obtiene conjunto de prueba
X_full_train, X_test, y_full_train, y_test= train_test_split(
    final_df.pro_X, final_df.y, train_size=proporcion_total_entrenamiento_prueba, 
    random_state=seed_)

# obtiene conjunto de entrnamiento y validacion
X_train, X_val, y_train, y_val  = train_test_split(
    X_full_train, y_full_train, train_size=proporcion_entrenamiento_validacion, 
    random_state=seed_)

# guardar los conjuntos en formato csv
conjuntos_dir = 'conjuntos'
os.makedirs(conjuntos_dir, exist_ok=True)
guarda_csv = lambda X, y, filename: pd.DataFrame({
    'pro_X': X, 'y': y
}).to_csv(f'{conjuntos_dir}/{filename}.csv', index=None)
guarda_csv(X_train, y_train, 'entrenamiento')
guarda_csv(X_val, y_val, 'validacion')
guarda_csv(X_test, y_test, 'prueba')

## Modelos de aprendizaje de máquinas


### Modelos no paramétricos

#### Representación por conteo de ocurrencias.
Dado que se busca resolver un problema de clasificación de documentos, es necesario representar los documentos de forma numérica. A continuación usaremos la representación por conteo de apariciones de cada uno de los *tokens* presentes en el *vocabulario*.

Para realizar esta vectorización de documentos se utiliza la clase `sklearn.feature_extraction.text.CountVectorizer`. El método `fit_transform` de esta clase  recibe como argumento un `iterable` de *documentos*, extrae el *vocabulario* de dicho `iterable` y retorna la matriz de número de ocurrencias de cada *token* del *vocabulario*, en cada uno de los documentos del `iterable`. En otras palabras, definiendo $\text{tf}(t_i, \mathbf{d}_j)$ como el número de apariciones del *token* $t_i$ en el *documento* $\mathbf{d}_j$, `~CountVectorizer.fit_transform` retorna la matriz $\mathbf{C}$ definida por:
$$\big(c_{i, j}\big) = \text{tf}(t_j, \mathbf{d}_i)$$

#### Naive Bayes

Se emplea el algoritmo de *Naive Bayes* como base de referencia para los modelos más complejos empleados posteriormente. Específicamente, se utiliza una instancia de la clase `sklearn.naive_bayes.MultinomialNB` que está diseñado para trabajar con las características del tipo conteo. 



##### Pregunta 2
1. Instancie `nb_pipe` como un objeto de la clase `sklearn.pipeline.Pipeline` con los componentes:
    - `~CountVectorizer` inicializado con `max_features=20000` los demás parámetros por defecto.
    - `~MultinomialNB` inicializado con los parámetros por defecto.  
    
   Posteriormente, ajuste `nb_pipe` en el conjunto de *entrenamiento union validación* y guarde el modelo resultante en la carpeta `modelos/nb_pipe.pk` como un archivo `pickle`. 

In [ ]:
nb_pipe = Pipeline([('CountVectorizer', CountVectorizer(max_features=20000)),
                                      ('MultinomialNB', MultinomialNB())])

nb_pipe.fit(X_full_train, y_full_train)

path = '/modelos/'
#path = '/content/drive/My Drive/U_Proyectos/DataScienceLab/Ejercicio3/modelos/'
os.makedirs(path, exist_ok=True)
pipe_file = open(path + 'nb_pipe.pk', 'wb')
pk.dump(nb_pipe, pipe_file)
pipe_file = open(path + 'nb_pipe.pk', 'rb')
nb_pipe = pk.load(pipe_file)

2. Defina la función `evalua_sklearn`  que reciba como argumentos:
    - `y_true`: np.array de una dimensión, conteniendo las etiquetas de cada una de las observaciones 
    - `y_pred`: np.array, con las etiquetas predichas por algún modelo de clasificación
    - `nombre_clasificador`: str, define el nombre de la carpeta donde los resultados son guardados. 
    
  Esta función debe:
    - Imprimir en pantalla los resultados de clasificación, mediante `sklearn.metrics.classification_report` con 4 dígitos de precisión. Además debe guardar dichos resultadoes  en la ruta `f'resultados/{nombre_clasificador}/reporte_clasificacion.txt'`.
    - Generar un gráfico con la *matriz de confusión* mediante el uso de `sklearn.metrics.confusion_matrix` y `seaborn.heatmap`. Además debe guardar dicho gráfico en la ruta `f'resultados/{nombre_clasificador}/mc.pdf'`.
    
  Pruebe esta función con la predicción de `nb_pipe` sobre el conjunto de *prueba*, usando `nombre_clasificador='nb_pipe'`.

In [ ]:
def evalua_sklearn(y_true, y_pred, nombre_clasificador):
    #path = 'resultados/' + nombre_clasificador + '/'
    path = 'resultados/' + nombre_clasificador + '/'
    os.makedirs(path, exist_ok=True)

    class_report = classification_report(y_true=y_true, y_pred=y_pred, digits=4)
    print(class_report)
    report_file = open(path + 'reporte_clasificacion.txt', 'w')
    report_file.write(class_report)
    report_file.close()

    cm = confusion_matrix(y_true=y_true, y_pred=y_pred, normalize='all')
    hm = sns.heatmap(cm, annot=True)
    hm.get_figure().savefig(path + 'mc.pdf')
    plt.show()

#print(nb_pipe.score(X_test, y_test))
y_pred = nb_pipe.predict(X_test)
evalua_sklearn(y_true=y_test, y_pred=y_pred, nombre_clasificador='nb_pipe')

- Compruebe que obtiene un *accuracy* y un promedio ponderado de *f1-score* superiores a .89.  

#### Representación Tf-idf

Para esta sección y la posterior se emplea la clase `sklearn.feature_extraction.text.TfidfVectorizer`. 

Para comprender el algoritmo ***tf-idf*** (*term frequency - inverse document frequency*) necesitamos definir 3 de sus componentes:
- $\text{tf}(t_j, \mathbf{d}_i)$ que representa el número de apariciones del *token* $t_j$ en el *documento* $\mathbf{d}_i$.

- $\text{df}(t_j)$ que representa el número de documentos en que aparece el *token* $t_j$.
- $\text{idf}(t_j)$ que representa el inverso de la frecuencia del *token* $t_j$ en los *documentos* del *corpus*. En el caso particular del objeto `~.TfidfVectorizer` empleado, denotando por $n$ al numero de documentos del *corpus*, se usa una versión suavizada de la función $\text{idf}$. Esto es: 

$$
\text{idf}(t_j) = \log(\frac{1 + n}{1 + \text{df}(t_j)}) + 1
$$


Con dichos componentes, podemos definir a la representación vectorial del documento $\mathbf{d}_i$ mediante el vector $\textbf{tf-idf}(\mathbf{d}_i)$, cuyas coordenadas se calculan de la siguiente manera:

$$
\left(\textbf{tf-idf}(\mathbf{d}_i)_i\right) = \text{tf}(t_j, \mathbf{d}_i) \times \text{idf}(t_j)
$$


Así la salida del objeto `~.TfidfVectorizer` es la matriz de la concatenación por filas, de los vectores $\textbf{tf-idf}(\mathbf{d}_i)$ normalizados según la norma euclidiana, es decir, la matriz $\mathbf{X}$, definida por:


$$
\left( x_{i,j} \right) = \frac{\text{tf}(t_j, \mathbf{d}_i) \times \text{idf}(t_j)}{{||\textbf{tf-idf}(\mathbf{d}_i)||}_2} 
$$



donde ${||\cdot||}_2$ representa la norma euclidiana. En consecuencia se logra una representación vectorial sobre la esfera unitaria de la norma ${||\cdot||}_2$.

Una interpretación posible de esta representación vectorial es que cada *token* $t_j$ tiene mayor importancia en el documento $\mathbf{d}_i$:

- En la medida en que esta aparezca más veces en el documento
- En la medida en que esta aparezca menos veces en los demás documentos del corpus

#### Maquinas de soporte vectorial (SVM)

Se emplea el algoritmo de máquinas de soporte vectorial (*Support Vector Machines - SVM*) sobre la representación vectorial *tf-idf*. La clase utilizada para generar este modelo es `sklearn.svm.SVC`.



##### Pregunta 3
1. Instancie `svm_pipe` como un objeto de la clase `sklearn.pipeline.Pipeline` con los componentes:
    - `~.TfidfVectorizer` inicializado con `max_features=20000` los demás parámetros por defecto.
    - `~.SVC` inicializado con el kernel RBF y los parámetros por defecto.  
    
   Posteriormente, ajuste `svm_pipe` en el conjunto de *entrenamiento union validación* y guarde el modelo resultante en la carpeta `modelos/svm_pipe.pk` como un archivo `pickle`. Finalmente, reporte el desempeño del clasificador mediante `evalua_sklearn` con la predicción de `svm_pipe` sobre el conjunto de *prueba*, usando `nombre_clasificador='svm_pipe'`.

In [ ]:
svm_pipe = Pipeline([('TfidfVectorizer', TfidfVectorizer(max_features=20000)),
                                      ('SVM', SVC(kernel='rbf'))])

svm_pipe.fit(X_full_train, y_full_train)

path = '/modelos/'
#path = '/content/drive/My Drive/U_Proyectos/DataScienceLab/Ejercicio3/modelos/'
pipe_file = open(path + 'svm_pipe.pk', 'wb')
pk.dump(svm_pipe, pipe_file)
pipe_file = open(path + 'svm_pipe.pk', 'rb')
svm_pipe = pk.load(pipe_file)

y_pred = svm_pipe.predict(X_test)
evalua_sklearn(y_true=y_test, y_pred=y_pred, nombre_clasificador='svm_pipe')

Las máquinas de soporte vectorial son algoritmos que pueden ser muy sensibles a los hiperparámetros. Por esta razón es útil emplear un esquema de validación cruzada. A continuación se implementa un esquema de validación cruzada simple, que explora sólo diferentes kernels y coeficientes de regularización.
2. Instancie `svm_grid` como un objeto de la clase `sklearn.model_selection.GridSearchCV`con los parámetros:
    - `n_jobs=-1` para usar todos los núcleos disponibles
    - `param_grid` definido de tal forma que le permita probar las combinaciones de los siguientes hiperparámetros de `~SVC`:
        - `kernel` en {`'lineal'`, `'rbf'`}
        - `C` en {`.01`, `.1`, `1`, `10`, `100`}

    - `cv=3` para generar un esquema de validación cruzada estratificada con 3 *fold*.
    - `verbose=1` para reportar el progreso del ajuste en pantalla
    
  Los demás parámetros quedan con sus valores por defecto.  
  
  Ajuste `grid_search_svm` usando `svm_pipe` sobre el conjunto *entrenamiento union validación* y guarde el modelo resultante en la carpeta `modelos/svm_grid.pk` como un archivo `pickle`. Luego reporte el desempeño del clasificador mediante `evalua_sklearn` con la predicción de `svm_grid` sobre el conjunto de *prueba*, usando `nombre_clasificador='svm_grid'`.  
  
  **Obs**: Evite fuga de información al combinar ``svm_pipe`` con `grid_search_svm`.

  Finalmente, en ruta `resultados/svm_grid/mejores_parametros.txt` guarde los mejores parámetros obtenidos en `svm_grid`.

In [ ]:
# no olvide fijar la semilla 
np.random.seed(seed_)

svm_pipe = Pipeline([('TfidfVectorizer', TfidfVectorizer(max_features=20000)),
                                      ('SVM', SVC())])
parameters = {'SVM__kernel':('lineal', 'rbf'), 'SVM__C':[.01, .1, 1, 10, 100]}
svm_grid = GridSearchCV(estimator = svm_pipe, param_grid=parameters, n_jobs=-1, cv=3, verbose=1)

svm_grid.fit(X_full_train, y_full_train)

y_pred = svm_grid.predict(X_test)
evalua_sklearn(y_true=y_test, y_pred=y_pred, nombre_clasificador='svm_grid')

print(svm_grid.best_params_)
path = 'resultados/svm_grid/'
# path = '/content/drive/My Drive/U_Proyectos/DataScienceLab/Ejercicio3/resultados/svm_grid/'
with open(path + 'mejores_parametros.txt', 'w') as file:
    file.write(str(svm_grid.best_params_))

### Modelos paramétricos

En lo que sigue del ejercicio se construirá una red neuronal recurrente (*RNN*). Para ello se usará principalmente la librería `torchtext` que provee una serie de herramientas que facilitan el manejo de texto para redes neuronales.

#### Representación vectorial densa

En las representaciones vectoriales anteriores, cada *token* del *vocabulario* constituye una dimension del espacio, es decir, hay tantas dimensiones en el espacio vectorial de representación como *tokens* en el *vocabulario*.

Hay una desventaja enorme en esta representación, además de la cantidad enorme de dimensiones que genera. Esta es que básicamente trata todos los *token* como entidades independientes sin relación entre sí. Lo que se busca en una representación densa es alguna noción de similitud entre las palabras.

Por ejemplo, supoga que se construye un modelo de lenguaje y que hemos visto las oraciones:

- El matemático corrió a la tienda.
- El físico corrió a la tienda.
- El matemático resolvió un problema abierto.

en el conjunto de *entrenamiento*. Ahora supongamos que obtenemos una nueva oración no presente en el conjunto de *entrenamiento*:

- El físico resolvió un problema abierto.

Si bien el modelo de lenguaje puede funcionar bien en esta oración, sería mejor si se pudiera utilizar los siguientes aspectos:

- Se ha observado matemático y físico en el mismo papel en una oración. De alguna manera tienen una relación semántica.
- Se ha observado al matemático en el rol del físico en una oración análoga a esta nueva oración.

Así, se podría inferir que el físico en realidad encaja bien en la nueva oración. Esto esconde una noción de similitud: queremos decir similitud semántica, no simplemente tener representaciones ortográficas similares. Este ejemplo, por supuesto, se basa en una suposición lingüística fundamental: que las palabras que aparecen en contextos similares están relacionadas semánticamente entre sí. Esto se llama hipótesis ***distribucional***.

Sobre dicha hipotesis se basa la construcción de modelos de procesamiento de lenguaje natural llamado ***word embeddings*** que se utilizan a continuación.

<center>Explicación adaptada de <a href="https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html">Word Embedding: Encoding Lexical Semantics - Pytorch</a></center> 




##### Pregunta 4 
La forma de cargar datos en formato csv que provee `torchtext`, es mediante los objetos `torchtext.data.dataset.TabularDataset`. A direfencia de `pandas.read_csv`, donde la función puede inferir el tipo de dato de cada columna, `~.TabularDataset` necesita que dichos tipos de dato sean declarados, mediante el objeto `torchtext.data.Field`.

1. Instancie `train_td`, `val_td` y `test_td` mediante el método `~.TabularDataset.splits` que le permita cargar textos de los archivos `conjuntos/*.csv` respectivos. Para ello tendrá que instanciar dos objetos de la clase `~.Field` que definan el tipo de dato para cada columna:

- La columna `y`: declarada mediante el objeto `etiqueta_fd` como instancia de `~.Field`, con parametros `sequential=False, use_vocab=False, batch_first=True, dtype=torch.float` y los demás por defecto.
- La columna `pro_X`:  declarada mediante el objeto `documento_fd`, como instancia de `~.Field`, con parámetros `include_lengths=True, batch_first=True` y los demás por defecto. El primer parámetro (`include_lengths=True`) implica que cada *documento* cargado de la columna se entregará en forma de tupla, donde además de la secuencia de *tokens* de aquel *documento*, se adjuntará el largo de la secuencia o número de tokens contenidos en este.

**Obs**: Tendrá que inferir los parámetros con los cuales emplear el método `~.splits()`.

In [ ]:
documento_fd = Field(include_lengths = True,
                     batch_first = True)
etiqueta_fd = Field(sequential = False,
                    use_vocab = False,
                    batch_first = True,
                    dtype = torch.float)

In [ ]:
train_td, val_td, test_td = TabularDataset.splits(
    path = conjuntos_dir+'/',
    train = 'entrenamiento.csv',
    validation = 'validacion.csv',
    test = 'prueba.csv',
    format = 'csv',
    fields = [('pro_X', documento_fd),
              ('y', etiqueta_fd)],
              skip_header=True
)

`torchtext` también provee objetos que permiten iterar sobre un objeto de tipo `~.TabularDataset`. En particular, en el procesamiento de texto es beneficioso generar *batches* de secuencias que tengan largo similar, para ahorrar tiempo de escritura, ya que dichos *batches* al tener forma tensorial, deben ser completados por ceros (*padding*). Así por ejemplo, las secuencias:
```
[ [4, 16, 3, 8],
  [5, 2],
  [6, 6, 7, 9, 2] ]
```
deben ser transformadas a:
```
[ [4, 16, 3, 8, 0],
  [5, 2, 0, 0, 0],
  [6, 6, 7, 8, 2] ]
```

Mediante la clase ` torchtext.data.BucketIterator`, es posible iterar sobre instancias de `~.TabularDataset`, de manera que se minimiza la cantidad de *padding* y al mismo tiempo se mantiene un orden aleatorio de los datos.

2.  Instancie *iterators* de cada uno de los `~.TabularDataset` instanciados, mediante `~.BucketIterator.splits` con los parámetros `batch_size=32, sort_key=lambda x: len(x.pro_X),
device=device, sort=True, sort_within_batch=True`.


In [ ]:
train_iter, val_iter, test_iter = BucketIterator.splits(
    (train_td, val_td, test_td), batch_size = 32,
    sort_key=lambda x: len(x.pro_X), device = device,
    sort = True, sort_within_batch = True)

3. Obtenga la representación vectorial del vocabulario con el *word embedding* GloVe entrenado en [Wikipedia 2014 + GigaWord 5](https://nlp.stanford.edu/projects/glove/). Para ello:
- Instancie el objeto `glove` como una instancia de la clase `torchtext.vocab.GloVe` con los parámetros`name="6B", dim=300`.
- Ejecute el método `build_vocab` del objeto `documento_fd` con `max_size=20000` y los demás parámetros que correspondan   
  Compruebe posteiormente que al llamar a `documento_fd.vocab.vectors.shape` obtiene las dimensiones `(20002,300)`

In [ ]:
glove = GloVe(name = '6B',
              dim = 300)

In [ ]:
documento_fd.build_vocab(train_td, vectors = glove, max_size = 20000)

In [ ]:
documento_fd.vocab.vectors.shape

#### LSTM 
Finalmete se implementa una LSTM bidireccional. La estructura de la red LSTM ya fue revisada en el material del curso, sin embargo, no se estudió su variante bidireccional. Esta denominación implica que la secuencia de tokens presente en cada texto es procesada desde el primer token hasta el último en una LSTM y desde el úĺtimo hasta el primero en otra LSTM, tal como lo ilustra la siguiente imagen.

<center> <img src="http://colah.github.io/posts/2015-09-NN-Types-FP/img/RNN-bidirectional.png" align="middle"> </center>
<center> Fuente: <a href="http://colah.github.io/posts/2015-09-NN-Types-FP/"> Colah's Blog </a></center>

Por consistencia con la imagen, supongamos que la LSTM bidireccional recibe un documento compuesto por la secuencia de tokens $x_0, \ldots, x_u$, de largo $i+1$. Sean $\text{LSTM}$ la RNN que recibe los inputs desde $0$ hasta $i$ y $\text{LSTM}'$ la RNN que los recibe desde $i$ hasta $0$. A cada input de la secuencia, $x_j$, corresponde un *output*, $y_j$ que consiste en la concatenación $\left(A(S_j, x_j),A'(S'_{i-j}, x_j)\right)$. Lo que debe ser considerado como la salida final de la LSTM bidireccional es la primera mitad de $y_i$ y la segunda mitad de $y_0$ pues corresponden a $A(S_i, x_i)$ y $A'(S'_i, x_0)$ respectivamente. Observe que este es output se asocia a la secuencia completa.

La LSTM bi-direccional que se implementa está diseñada para usar un *word embedding* fijo, como el que fue calculado en la sección anterior.

La estructura de la red es la siguiente:
- *Word embedding* pre-entrenado (no deben calcularse gradientes en esta sección de la red), implementado con la clase `torch.nn.Embedding`.
- LSTM bidireccional, tomando como salida la concatenación recién explicada e implementada con la clase `torch.nn.LSTM` con los parámetros `batch_first=True` y `bidirectional=True`. 
- Dropout con probabilidad .5
- Capa totalmente conectada con salida de tamaño 1 y función de activación sigmoide.

Dado que `~.BucketIterator` entrega un tensor con *paddings* de cero, es necesario transformar el input de `~.LSTM`, de tal forma que esta no procese los ceros del tensor de entrada. Por esta razón al pasar del output de `~.Embedding`, al input de `~.LSTM`, es necesario emplear la función `torch.nn.utils.rnn.pack_padded_sequence` que permite transformar una secuencia con *paddings* de 0, en una secuencia que los oculta. Dicha función es capaz de transformar un input de la forma:
```
seq = torch.tensor([[4,5,6], [1,2,0], [3,0,0]])
lens = [3, 2, 1]
packed = pack_padded_sequence(seq, lens, batch_first=True, enforce_sorted=True)
```
donde el objeto `packed` tendrá la forma:
```
PackedSequence(data=tensor([4, 1, 3, 5, 2, 6]), batch_sizes=tensor([3, 2, 1]), sorted_indices=None, unsorted_indices=None)
```
y estará listo para ser procesado como entrada de `~.LSTM`.

Sin embargo, el empleo de aquella función, obliga a usar su función inversa, `torch.nn.utils.rnn.pad_packed_sequence`, sobre la salida de `~.LSTM` y así que pasar de un objeto `PackedSequence` a su formato tensorial con *padding* de 0. En consecuencia al emplear:
```
lstm = nn.LSTM(...)
packed_output, _ = lstm(packed)
seq_unpacked, lens_unpacked = pad_packed_sequence(packed, batch_first=True)
``` 
Esto permitirá encontrar el output de la LSTM bidireccional en el tensor `seq_unpacked` de tamaño $B \times T \times C$, donde $B$ representa el tamaño del *batch*, $T$ el largo de la secuencia más larga del *batch* y $C$ el tamaño de la dimension del espacio de carácteristicas de la salida de `~.LSTM`. 


##### Pregunta 5

1. Defina la clase `Glove6B300BiLSTM` heredando de `torch.nn.Module` y definiendo los métodos:
- `__init__`: recibe como parametros `hidden_size`, que regula el parámetro homónimo de `torch.nn.LSTM`; y `text_field` que apunta al `torchtext.data.Field` que ya contiene la representación vectorial densa del *vocabulario* que es utilizado en la capa del *word embedding*.  
- `forward`: recibe como argumentos `text` y `text_len` que corresponden a los objetos que entrega `BucketIterator`.  
Haga uso de las funciones `~.pack_padded_sequence` con parámetros `batch_first=True, enforce_sorted=True` y `~.pad_packed_sequence`, con parámetro `batch_first=True`. Sea ciudadoso en la selección de los segmentos del tensor de salida de `~.pad_packed_sequence` que deben ser considerados para las capas posteriores.   

*Hint*: note que para construir el output de la $\text{LSTM}$ bidireccional, deberá seleccionar segmentos de un vector de salida y concatenarlos de manera conveniente. Para ello le será de ayuda la variable `text_len`.

  
Instancie `modelo` como objeto de la clase `Glove6B300BiLSTM` con `hidden_size=128` y `text_field=documento_fd`. Recuerde instanciarlo en el espacio de memoria adecuado mediante el método `to(device)`.

In [ ]:
# Definir la red
class Glove6B300BiLSTM(torch.nn.Module):
    def __init__(self, 
                 hidden_size, 
                 text_field):
        super().__init__()
        # Capa de embedding
        self.embedding = nn.Embedding.from_pretrained(
            text_field.vocab.vectors.clone(),
            freeze = True
        )
        # Capa LSTM
        self.lstm = nn.LSTM(text_field.vocab.vectors.shape[1],
                           hidden_size,
                           bidirectional = True)
        # Capa de salida
        self.fc = nn.Linear(hidden_size * 2,
                            out_features = 1)
        # Dropout
        self.dropout = nn.Dropout(0.5)
        
        # Sigmoide
        self.sigm = nn.Sigmoid()
        
        # HiddenSize
        self.hidden_size = hidden_size

    def forward(self, text, text_len):

        embedded = self.embedding(text)

        packed = pack_padded_sequence(embedded, text_len, batch_first=True, enforce_sorted=True)

        packed_output, _ = self.lstm(packed)
        
        seq_unpacked, lens_unpacked = pad_packed_sequence(packed_output, batch_first=True)
        
        seq_unpacked_out_f = seq_unpacked[:, -1, :self.hidden_size]
        
        seq_unpacked_out_r = seq_unpacked[:, 0, self.hidden_size:]
        
        output = torch.cat((seq_unpacked_out_f, seq_unpacked_out_r), dim = 1)

        predictions = self.fc(self.dropout(output))

        return self.sigm(predictions).view(-1)

In [ ]:
modelo = Glove6B300BiLSTM(hidden_size= 180, text_field = documento_fd).to(device)

2. Implemente el ciclo de entrenamiento de la red `Glove6B300BiLSTM`. Para ello use:
- Entropía cruzada binaria como funcion de costo - `torch.nn.BCELoss()`
- Adam cómo algoritmo de optimización, con `lr=2e-4`
- 10 epocas de entrenamiento
- Los `BucketIterator` definidos en la pregunta anterior para recorrer los conjuntos de entrenamiento y validación.
- Guarde en una lista el historico de valores de la función de costo en el conjunto de entrenamiento y de validación
- Al final de cada época guarde el modelo mediante `guardar_modelo` en la ruta `modelos/Glove6B300BiLSTM.h5`, si es que la función de costo sobre el conjunto de validación es menor que la menor función de costo sobre el conjunto de validación observada en épocas anteriores. 

Al finalizar el entrenamiento, muestre en pantalla un gráfico con el historico de la función de costo en el conjunto de entrenamiento y en el de validación y guardelo en `resultados/Glove6B300BiLSTM/costo_historico.pdf`.


In [ ]:
def save_model_all(model, save_dir, model_name):
    """
    :param model:  nn model
    :param save_dir: save model direction
    :param model_name:  model name
    :param epoch:  epoch
    :return:  None
    """
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    save_prefix = os.path.join(save_dir, model_name)
    save_path = '{}.h5'.format(save_prefix)
    print("save all model to {}".format(save_path))
    output = open(save_path, mode="wb")
    torch.save(model.state_dict(), output)
    torch.save(model.state_dict(), save_path)
    output.close()

In [ ]:
def register(res_list):
    '''Obtiene la perdida promedio y su desviacion estandar en batches.'''
    
    losses, nums = zip(*res_list)
    
    N = np.sum(nums)
    loss_mean = np.sum(np.multiply(losses, nums))/N
    loss_std = np.sqrt(np.sum(np.multiply((losses-loss_mean)**2, nums))/(N-1))
    
    return loss_mean, loss_std

In [ ]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb[0], xb[1]), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

In [ ]:
def fit(epochs,
        model,
        loss_func,
        opt,
        train_dl,
        valid_dl,
        metric=None,
        only_print=True,
        print_leap = 1):

    if metric is None:
        metric = loss_func

    learning_data = pd.DataFrame(
        columns=['epoch', 'train_mean', 'train_std', 'val_mean', 'val_std'])
    
    best_loss = np.inf

    for epoch in range(epochs):

        # Entrenamiento -------------------------------------------------------
        train_res = []
        model.train()

        for x in train_dl:
            # Para entrenar se usa la funcion de perdida
            loss_batch(model, loss_func, x.pro_X, x.y, opt)

            # Para almacenar se puede usar una metrica
            train_res.append(loss_batch(model, metric, x.pro_X, x.y))

        # Validacion ----------------------------------------------------------
        # Para evaluar se puede utilizar un metrica de rendimiento
        model.eval()

        with torch.no_grad():
            val_res = [
                loss_batch(model, metric, x.pro_X, x.y) for x in valid_dl
            ]

        val_loss, val_std = register(val_res)
        train_loss, train_std = register(train_res)

        if val_loss < best_loss:
            save_model_all(model, 'modelos', 'Glove6B300BiLSTM', '')
            best_loss = val_loss
        
        if epoch % print_leap == 0:
            print('Epoca:', epoch, '- val:', val_loss, '- train:', train_loss)

        learning_data = learning_data.append(
            {
                'epoch': epoch,
                'train_mean': train_loss,
                'train_std': train_std,
                'val_mean': val_loss,
                'val_std': val_std
            },
            ignore_index=True)

    if only_print:
        print('Proceso terminado')
    else:
        return learning_data

In [ ]:
loss_func = nn.BCELoss()
opt = optim.Adam(modelo.parameters(), lr = 2e-4)
epochs = 10

In [ ]:
learning_data = fit(epochs,
                    modelo,
                    loss_func,
                    opt,
                    train_iter,
                    val_iter,
                    only_print=False,
                    print_leap=2)

In [ ]:
def show_learning_curves(learning_data, leaps = 1):
    '''Genera curvas de aprendizaje dado data dataframe resultado de fit().'''
    
    fig, ax = plt.subplots(figsize=[10, 7])
    ax.grid()


    epcs = learning_data['epoch'][::leaps]
    val_loss_lower = (learning_data['val_mean'] -
                      learning_data['val_std'])[::leaps]

    val_loss_upper = (learning_data['val_mean'] +
                      learning_data['val_std'])[::leaps]

    train_loss_lower = (learning_data['train_mean'] -
                        learning_data['train_std'])[::leaps]
    train_loss_upper = (learning_data['train_mean'] +
                        learning_data['train_std'])[::leaps]

    ax.plot(epcs,
            learning_data['val_mean'][::leaps],
            'o--',
            color="g",
            label="Validation")

    ax.fill_between(epcs, val_loss_lower, val_loss_upper, alpha=0.1, color='g')

    ax.plot(epcs,
            learning_data['train_mean'][::leaps],
            'o--',
            color="r",
            label="Train")

    ax.fill_between(epcs, train_loss_lower, train_loss_upper, alpha=0.1, color='r')

    ax.set_title('Curva de Aprendizaje', fontsize=25)
    ax.set_xlabel('Epocas', fontsize=15)
    ax.set_ylabel('Loss', fontsize=15)
    ax.legend()

In [ ]:
show_learning_curves(learning_data)

3. Obtenga la predicción de `modelo` sobre el conjunto de prueba y reporte el desempeño del clasificador mediante `evalua_sklearn` con la predicción de `modelo` sobre el conjunto de *prueba*, usando `nombre_clasificador='Glove6B300BiLSTM'`.

In [ ]:
modelo.load_state_dict(torch.load('/modelos/Glove6B300BiLSTMepoch.h5'))
modelo.eval()
trues = []
preds = []
with torch.no_grad():
    for batch in test_iter:
        t, tl = batch.pro_X
        trues.append(batch.y)

        preds.append(modelo(t, tl).squeeze()) 

y_pred = torch.cat(preds).cpu()
y_true = torch.cat(trues).cpu()

In [ ]:
evalua_sklearn(y_true, y_pred>0.5, 'Glove6B300BiLSTM')